In [1]:
import pandas as pd
import snowflake.connector

In [2]:
conn = snowflake.connector.connect(user = "mdimasaac", password = "Qaz1234.", account = "vliobpf-yr09142",
                                    database = "store_it", schema = "store_schema")
cur = conn.cursor()

In [3]:
qdf = "select * from store_data"
cur.execute(qdf)
df = cur.fetch_pandas_all()
print(df.shape)
df.head(1)

(10000, 12)


,DATE,MONTH,YEAR,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
0,2022-10-02,10,2022,OFF-ST-10000988,"Fellowes Folders, Blue",Storage,25.99,31.97,3,77.97,5.98,17.94


In [4]:
len(df["CATEGORY"].unique().tolist())

17

### skip this


In [3]:
qdflog = "select * from logistics"
cur.execute(qdflog)
dflog = cur.fetch_pandas_all()
print(dflog.shape)
dflog.head()

(1856, 7)


,PRODUCT_NAME,PRODUCT_ID,CATEGORY,BUY_PRICE,SELL_PRICE,STOCK,PROFIT_PER_ITEM
0,"Fellowes Folders, Blue",OFF-ST-10000988,Storage,25.99,28.59,3,2.60
1,"SanDisk Numeric Keypad, Bluetooth",TEC-AC-10004144,Accessories,55.99,68.87,15,12.88
2,"Avery Legal Exhibit Labels, 5000 Label Set",OFF-LA-10001915,Labels,11.99,14.75,7,2.76
3,"Fellowes Folders, Wire Frame",OFF-ST-10004550,Storage,24.99,29.24,13,4.25
4,"Memorex Memory Card, USB",TEC-AC-10004068,Accessories,101.99,128.51,6,26.52


In [22]:
dfcart = pd.DataFrame(columns=["Product Name","Price","Qty","Total"])
dfcart.to_csv("cart.csv",index = False)

In [23]:
cart = pd.read_csv("cart.csv")
cart.head()

,Product Name,Price,Qty,Total


In [19]:

dfcopy = dflog.copy()
dfcopy["STOCK"][dfcopy["PRODUCT_NAME"] == "Fellowes Folders, Blue"] = dfcopy["STOCK"][dfcopy["PRODUCT_NAME"] == "Fellowes Folders, Blue"] - 1

C:\Users\mdima\AppData\Local\Temp\ipykernel_6928\4046239720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcopy["STOCK"][dfcopy["PRODUCT_NAME"] == "Fellowes Folders, Blue"] = dfcopy["STOCK"][dfcopy["PRODUCT_NAME"] == "Fellowes Folders, Blue"] - 1


In [20]:
dfcopy["STOCK"][dfcopy["PRODUCT_NAME"] == "Fellowes Folders, Blue"].values[0]

2

In [31]:
df1 = df[df["PRODUCT_NAME"] == "Fellowes Folders, Blue"].head(1)
df1["QUANTITY"][df1["PRODUCT_NAME"]=="Fellowes Folders, Blue"] = df1["QUANTITY"][df1["PRODUCT_NAME"]=="Fellowes Folders, Blue"]-1
df1["SALES"]=df1["QUANTITY"]*df1["SELL_PRICE"]
df1["PROFIT"]=df1["PROFIT_PER_ITEM"]*df1["QUANTITY"]
df1

,DATE,MONTH,YEAR,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
0,2022-10-02,October,2022,OFF-ST-10000988,"Fellowes Folders, Blue",Storage,25.99,28.59,2,57.18,2.6,5.2


In [38]:
conn = snowflake.connector.connect(user = "mdimasaac", password = "Qaz1234.", account = "vliobpf-yr09142",
                                    database = "store_it", schema = "store_schema")
cur = conn.cursor()
qdf = pd.DataFrame()
for name in dfcopy["CATEGORY"].unique().tolist():
# name = 'Fellowes Folders, Blue'
    q_df = "SELECT * FROM STORE_DATA WHERE CATEGORY = '"+name+"' LIMIT 1"
    cur.execute(q_df)
    qdf1 = cur.fetch_pandas_all()
    qdf = pd.concat([qdf,qdf1], axis = 0)
qdf.head()

2023-04-07 21:39:55.700 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 3.0.2, Python Version: 3.8.0, Platform: Windows-10-10.0.19041-SP0
2023-04-07 21:39:55.703 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2023-04-07 21:39:57.154 INFO    snowflake.connector.cursor: query: [SELECT * FROM STORE_DATA WHERE CATEGORY = 'Storage' LIMIT 1]
2023-04-07 21:39:57.352 INFO    snowflake.connector.cursor: query execution done
2023-04-07 21:39:57.355 INFO    snowflake.connector.cursor: Number of results in first chunk: 1
2023-04-07 21:39:57.359 INFO    snowflake.connector.cursor: query: [SELECT * FROM STORE_DATA WHERE CATEGORY = 'Accessories' LIMIT 1]
2023-04-07 21:39:57.512 INFO    snowflake.connector.cursor: query execution done
2023-04

,DATE,MONTH,YEAR,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
0,2022-10-02,October,2022,OFF-ST-10000988,"Fellowes Folders, Blue",Storage,25.99,28.59,3,77.97,2.60,7.80
0,2022-10-02,October,2022,TEC-AC-10004144,"SanDisk Numeric Keypad, Bluetooth",Accessories,55.99,68.87,7,391.93,12.88,90.16
0,2022-10-02,October,2022,OFF-LA-10001915,"Avery Legal Exhibit Labels, 5000 Label Set",Labels,11.99,14.75,3,35.97,2.76,8.28
0,2022-10-02,October,2022,TEC-PH-10003963,"Apple Signal Booster, Full Size",Phones,137.99,157.31,3,413.97,19.32,57.96
0,2020-05-08,May,2020,TEC-CO-10004325,"Canon Personal Copier, Laser",Copiers,142.99,174.45,3,428.97,31.46,94.38


----------

### and start from this

In [7]:
import plotly.express as px
df.to_csv("store_data_snowflake.csv")

In [8]:
# df["Y-M"] = pd.to_datetime(df.YEAR.astype(str) + '-' + df.MONTH.astype(str))
# df['MONTH'] = pd.to_datetime(df['MONTH'], format='%m').dt.strftime('%B')

df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime('%b %Y')
df = df.sort_values(by="DATE")
df['DATE'] = pd.to_datetime(df['DATE'], format='%b %Y')
df = df.reset_index(drop=True)
df.head(1)

,DATE,MONTH,YEAR,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
0,2019-04-01,4,2019,OFF-ST-10003835,"Rogers Folders, Single Width",Storage,27.99,30.79,1,27.99,2.8,2.8


In [ ]:
# profit sum, quantity sum, sales sum

In [20]:
df[["CATEGORY","PROFIT"]][(df["MONTH"] == "4") & (df["YEAR"] == 2019)].groupby(["CATEGORY"]).sum()["PROFIT"]

CATEGORY
Accessories     104.94
Appliances       27.70
Art             404.87
Binders         294.68
Bookcases      1096.42
Chairs          254.00
Copiers        1192.20
Envelopes        23.79
Fasteners        32.88
Furnishings      89.92
Labels           38.08
Machines        107.64
Paper            79.06
Phones           66.20
Storage         979.42
Supplies        150.01
Name: PROFIT, dtype: float64

In [9]:
import plotly.express as px
import plotly.graph_objects as go

data = []
month = ["4","5","6"]
year = [2019]
m_name = ["Apr","May","Jun"]
x = df["CATEGORY"].sort_values().unique().tolist()
for i in year:
    for j,k in list(zip(month,m_name)):
        data.append(go.Bar(name=k+" "+str(i), x=x,
             y=df[["CATEGORY","PROFIT"]][(df["MONTH"] == j) & (df["YEAR"] == i)].groupby(["CATEGORY"]).sum()["PROFIT"]))

fig = go.Figure(data=data)
fig.update_layout(barmode='group', template="plotly_dark")

fig.show()

In [49]:
sel = df[(df["CATEGORY"]=="Art") & (df["YEAR"]==2022) & (df["MONTH"]=="9")].sort_values(by="PROFIT", ascending = False)
top6 = sel.groupby(["PRODUCT_NAME"]).sum().sort_values(by="PROFIT", ascending = False).head(6).index.values.tolist()
prod = []
for i in sel["PRODUCT_NAME"]:
    if (i in top6):
        prod.append(i)
    else:
        prod.append("Others")
sel["PRODUCT_NAME"] = prod
selected = sel.groupby(["PRODUCT_NAME"]).sum().sort_values(by="PROFIT", ascending = False)
selected

C:\Users\mdima\AppData\Local\Temp\ipykernel_8612\1495815780.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\mdima\AppData\Local\Temp\ipykernel_8612\1495815780.py:10: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,YEAR,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
PRODUCT_NAME,,,,,,,
Others,68748,962.66,1177.25,110,2830.90,214.59,613.26
"Binney & Smith Canvas, Easy-Erase",4044,103.98,119.58,11,571.89,15.60,85.80
"Binney & Smith Pencil Sharpener, Fluorescent",4044,53.98,68.02,12,323.88,14.04,84.24
"Sanford Pencil Sharpener, Water Color",8088,103.96,132.04,12,311.88,28.08,84.24
"Boston Markers, Water Color",4044,59.98,76.78,10,299.90,16.80,84.00
"BIC Canvas, Easy-Erase",2022,53.99,65.33,7,377.93,11.34,79.38
"Boston Markers, Easy-Erase",6066,71.97,89.25,13,311.87,17.28,74.88


In [53]:
selected.PROFIT.values.round(2).tolist()

[613.26, 85.8, 84.24, 84.24, 84.0, 79.38, 74.88]

In [77]:
import plotly.express as px
import plotly.graph_objects as go

sel = df[(df["CATEGORY"]=="Phones") & (df["YEAR"]==2021) & (df["MONTH"]=="7")].sort_values(by="PROFIT", ascending = False)
top = sel.groupby(["PRODUCT_NAME"]).sum().sort_values(by="PROFIT", ascending = False).head(10).index.values.tolist()
prod = []
for i in sel["PRODUCT_NAME"]:
    if (i in top):
        prod.append(i)
    else:
        prod.append("Others")
sel["PRODUCT_NAME"] = prod
selected = sel.groupby(["PRODUCT_NAME"]).sum().sort_values(by="PROFIT", ascending = False)
labels = selected.index.values.tolist()
pull = []
for i in range(len(labels)):
    pull.append(0.1)
fig2 = go.Figure(go.Pie(
                    labels = labels,
                    values = selected.PROFIT.values.round(2).tolist(),hovertemplate = "%{label}<br>%{percent} <extra></extra>",
                    hole=.7,
                    marker_line_color='white',
                    marker_line_width=1.25,
                    pull= pull,
                    textinfo="label+percent"
                ))
fig2.update_layout(
                    autosize=True,  # Automatically adjust size to fit entire layout
                    margin=dict(l=0, r=0, t=0, b=0),)  # Remove margin around chart
fig2.update_layout(showlegend=False, template="plotly_dark")
fig2.update_layout(
font=dict(
size=16,))

fig2.show()
# selected = plotly_events(fig)

C:\Users\mdima\AppData\Local\Temp\ipykernel_8612\3978743886.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\mdima\AppData\Local\Temp\ipykernel_8612\3978743886.py:13: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [82]:
selected["PROFIT_PER_ITEM"][selected.index.values == "Samsung Smart Phone, VoIP"].values.tolist()[0]

152.64

In [90]:
dfline = df[df["PRODUCT_NAME"] == "Binney & Smith Canvas, Easy-Erase"].sort_values(by="DATE", ascending = True)
dfline

,DATE,MONTH,YEAR,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BUY_PRICE,SELL_PRICE,QUANTITY,SALES,PROFIT_PER_ITEM,PROFIT
8376,2019-10-01,10,2019,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,59.79,2,103.98,7.8,15.6
5746,2021-03-01,3,2021,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,59.79,2,103.98,7.8,15.6
5797,2021-03-01,3,2021,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,59.79,3,155.97,7.8,23.4
283,2021-04-01,4,2021,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,59.79,3,155.97,7.8,23.4
954,2021-08-01,8,2021,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,59.79,4,207.96,7.8,31.2
1896,2021-12-01,12,2021,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,59.79,5,259.95,7.8,39.0
7133,2022-05-01,5,2022,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,59.79,3,155.97,7.8,23.4
9706,2022-09-01,9,2022,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,59.79,9,467.91,7.8,70.2
9984,2022-09-01,9,2022,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,59.79,2,103.98,7.8,15.6
3861,2023-01-01,1,2023,OFF-AR-10003633,"Binney & Smith Canvas, Easy-Erase",Art,51.99,59.79,1,51.99,7.8,7.8


In [92]:
fig = px.line(dfline, x="DATE", y="PROFIT", title='Life expectancy in Canada')
fig.update_layout(template = "plotly_dark")
fig.show()

In [93]:
dfline["PROFIT"].tolist()

[15.6, 15.6, 23.4, 23.4, 31.2, 39.0, 23.4, 70.2, 15.6, 7.8, 7.8, 23.4]

______

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [129]:
ml = df[["DATE","PRODUCT_NAME","CATEGORY","QUANTITY","PROFIT"]]
ml

,DATE,PRODUCT_NAME,CATEGORY,QUANTITY,PROFIT
0,2019-04-01,"Rogers Folders, Single Width",Storage,1,2.80
1,2019-04-01,"Green Bar Computer Printout Paper, 8.5 x 11",Paper,5,18.70
2,2019-04-01,"Ibico Binder Covers, Durable",Binders,9,27.00
3,2019-04-01,"Hon Shipping Labels, Laser Printer Compatible",Labels,3,6.27
4,2019-04-01,"Hewlett Copy Machine, Laser",Copiers,2,115.72
...,...,...,...,...,...
9995,2022-09-01,"Enermax Keyboard, Programmable",Accessories,2,18.70
9996,2022-09-01,"Hon Removable Labels, Laser Printer Compatible",Labels,4,12.00
9997,2022-09-01,"Memorex Memory Card, Erganomic",Accessories,8,96.00
9998,2022-09-01,"GlobeWeis Business Envelopes, Recycled",Envelopes,5,18.75


In [ ]:
# index = product_name, category

In [131]:
ml_sel = ml[ml["CATEGORY"]=="Accessories"]
ml_sel

,DATE,PRODUCT_NAME,CATEGORY,QUANTITY,PROFIT
61,2019-04-01,"Memorex Router, Programmable",Accessories,2,89.28
95,2019-04-01,"Memorex Flash Drive, Bluetooth",Accessories,3,15.66
121,2020-04-01,"SanDisk Router, Bluetooth",Accessories,5,294.40
130,2020-04-01,"SanDisk Router, Programmable",Accessories,1,67.08
136,2020-04-01,"Enermax Keyboard, Erganomic",Accessories,3,31.59
...,...,...,...,...,...
9941,2022-09-01,"Memorex Router, Bluetooth",Accessories,13,415.74
9962,2022-09-01,"Enermax Flash Drive, Erganomic",Accessories,1,10.53
9972,2022-09-01,"Belkin Router, USB",Accessories,2,103.60
9995,2022-09-01,"Enermax Keyboard, Programmable",Accessories,2,18.70


In [132]:
ml_timely = ml_sel.pivot_table(columns="DATE", values="PROFIT", aggfunc="sum", index=["PRODUCT_NAME"]).fillna(0)
ml_timely

DATE,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-02-01,...,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01
PRODUCT_NAME,,,,,,,,,,,,,,,,,,,,,
"Belkin Flash Drive, Bluetooth",0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,16.80
"Belkin Flash Drive, Erganomic",0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.00,0.0,0.00,0.00,26.40,0.00,0.00
"Belkin Flash Drive, Programmable",0.0,47.3,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.00,0.0,28.38,0.00,28.38,0.00,0.00
"Belkin Flash Drive, USB",0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.00,0.0,0.00,0.00,10.08,0.00,0.00
"Belkin Keyboard, Bluetooth",0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.00,0.0,21.00,0.00,0.00,31.50,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"SanDisk Numeric Keypad, USB",0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,41.34,0.0,0.00,0.00,0.00,0.00,13.78
"SanDisk Router, Bluetooth",0.0,0.0,0.0,0.0,235.52,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
"SanDisk Router, Erganomic",0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,76.20


In [133]:
ml_timely.index.values.tolist()[:5]

['Belkin Flash Drive, Bluetooth',
 'Belkin Flash Drive, Erganomic',
 'Belkin Flash Drive, Programmable',
 'Belkin Flash Drive, USB',
 'Belkin Keyboard, Bluetooth']

In [121]:
X = ml_timely.drop(columns = "2023-03-01")
y = ml_timely["2023-03-01"]
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.4, random_state = 43)
num_train = X_train.select_dtypes(np.number)
num_test = X_test.select_dtypes(np.number)
scaler = MinMaxScaler().fit(num_train)
cols = scaler.get_feature_names_out(input_features = num_train.columns)
num_train_scaled = scaler.transform(num_train)
num_test_scaled = scaler.transform(num_test)
num_train_scaled = pd.DataFrame(num_train_scaled, columns = cols)
num_test_scaled = pd.DataFrame(num_test_scaled, columns = cols)
X_train_processed = num_train_scaled.copy()
X_test_processed = num_test_scaled.copy()
import plotly.graph_objs as go
xvalues = ml_name.index.values.tolist()
yvalues = y_test
trace1 = go.Scatter(x=xvalues,y=yvalues,
                    mode = "lines", name = "actual profit")
regr = LinearRegression()
regr.fit(X_train_processed, y_train)
pred_test = regr.predict(X_test_processed)
mean_score_test = np.mean(cross_val_score(regr_left, X_test_processed, y_test, cv = 5))
